### Initializing connection

In [2]:
import pyodbc

server = 'ICHIM\SQLEXPRESS03'
database = 'TicketManagementSystem'
username = ''
password = ''

connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

conn = pyodbc.connect(connection_string)

### Queries

#### Selecting events from a given year and month

In [4]:
cursor = conn.cursor()
def get_events_by_year_month(year, month):
    query = 'SELECT * FROM Events WHERE YEAR(StartDate) = ? AND MONTH(StartDate) = ?'
    
    cursor.execute(query, (year, month))

    column_names = [column[0] for column in cursor.description]
    print(column_names[0], column_names[5], column_names[6])

    events = cursor.fetchall()
    return events

events = get_events_by_year_month(2023, 12)
print(events[1])
print(len(events))
for event in events[:10]:
    print(event[0], event[5], event[6])

ID StartDate EndDate
('1C44D0CC-1076-4810-9AD7-488ACDFB24F6', 'FB07A381-0862-4CF5-9555-BE325FF20676', '53C153DE-9FA2-4137-8AD7-B82A7274FE67', 'Business networking', 'Business networking', datetime.datetime(2023, 12, 15, 0, 0), datetime.datetime(2023, 12, 19, 0, 0), 0)
19
EC4A95F4-BC7B-4F6B-BADE-31B7925C756B 2023-12-07 00:00:00 2023-12-11 00:00:00
1C44D0CC-1076-4810-9AD7-488ACDFB24F6 2023-12-15 00:00:00 2023-12-19 00:00:00
6CE02BA1-13A3-4938-89E5-4E7A6941056D 2023-12-17 00:00:00 2023-12-21 00:00:00
322649ED-0673-4532-BDE7-4EC3919D7C14 2023-12-08 00:00:00 2023-12-12 00:00:00
39B97750-6F8F-4BAD-A9E0-57B9DBA007D3 2023-12-05 00:00:00 2023-12-08 00:00:00
5FE3FE57-077D-49A8-A679-6DB9455DFE6A 2023-12-17 00:00:00 2023-12-21 00:00:00
0C258DFE-F9CC-4DDF-9626-8326EB8D017F 2023-12-11 00:00:00 2023-12-14 00:00:00
3F69AFB0-9589-454F-AE48-88B5A9DD6BF6 2023-12-24 00:00:00 2023-12-27 00:00:00
A8D450C5-88A7-4656-8E81-94005F603228 2023-12-15 00:00:00 2023-12-17 00:00:00
90177A76-7CB5-4825-B9C3-AB4C0C4E61D

#### Get final winnings of an event

In [58]:
cursor = conn.cursor()
def get_ticket_categories_for_event(event):
    query = 'SELECT * FROM TicketCategories WHERE Event_ID = ?'
    cursor.execute(query, (event[0]))
    ticket_categories = cursor.fetchall()
    return ticket_categories

def get_total_winnings_for_event(event):
    ticket_categories = get_ticket_categories_for_event(event)
    winnings = 0
    for ticket_category in ticket_categories:
        query = 'SELECT * FROM Orders WHERE TicketCategories_ID = ?'
        cursor.execute(query, (ticket_category[0]))
        orders = cursor.fetchall()
        for order in orders:
            winnings += order[5]
    return winnings

def get_hours_for_event(event):
    start_date = event[5]
    end_date = event[6]
    time_difference = end_date - start_date
    total_hours = time_difference.days * 24 + time_difference.seconds // 3600
    return total_hours

def get_price_per_hour_for_event(event):
    query = 'SELECT PricePerHour FROM Venues WHERE ID = ?'
    cursor.execute(query, (event[1]))
    return cursor.fetchone()

def get_total_cost_for_event(event):
    hours = get_hours_for_event(event)
    price_per_hour = get_price_per_hour_for_event(event)[0] # first column of entry
    return hours * price_per_hour

def get_final_winnings_for_event(event):
    cost = get_total_cost_for_event(event)
    winnings = get_total_winnings_for_event(event)
    return winnings - cost

In [61]:
def print_prices(events):
    for event in events:
        total_winnings = get_total_winnings_for_event(event)
        total_cost = get_total_cost_for_event(event) 
        final_winnings = get_final_winnings_for_event(event)

        print(f'Event: {event[0]}, Total Winnings: {total_winnings}, Cost: {total_cost}, Final Winnings: {final_winnings}')
print_prices(events[:10])

Event: EC4A95F4-BC7B-4F6B-BADE-31B7925C756B, Total Winnings: 121500, Cost: 2688.00, Final Winnings: 118812.00
Event: 1C44D0CC-1076-4810-9AD7-488ACDFB24F6, Total Winnings: 209400, Cost: 4896.00, Final Winnings: 204504.00
Event: 6CE02BA1-13A3-4938-89E5-4E7A6941056D, Total Winnings: 306900, Cost: 4896.00, Final Winnings: 302004.00
Event: 322649ED-0673-4532-BDE7-4EC3919D7C14, Total Winnings: 322500, Cost: 17299.20, Final Winnings: 305200.80
Event: 39B97750-6F8F-4BAD-A9E0-57B9DBA007D3, Total Winnings: 86850, Cost: 3672.00, Final Winnings: 83178.00
Event: 5FE3FE57-077D-49A8-A679-6DB9455DFE6A, Total Winnings: 276950, Cost: 4895.04, Final Winnings: 272054.96
Event: 0C258DFE-F9CC-4DDF-9626-8326EB8D017F, Total Winnings: 176650, Cost: 7221.60, Final Winnings: 169428.40
Event: 3F69AFB0-9589-454F-AE48-88B5A9DD6BF6, Total Winnings: 97650, Cost: 5767.20, Final Winnings: 91882.80
Event: A8D450C5-88A7-4656-8E81-94005F603228, Total Winnings: 264050, Cost: 4814.40, Final Winnings: 259235.60
Event: 90177A